In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from itertools import product

In [2]:
# Demand
df_demand = pd.read_excel('10node.xlsx', 'Customer')
df_demand = df_demand['Demand'].tolist()


In [3]:
# Fixed cost
f = 200

In [5]:
# Transportation cost
df_dis = pd.read_excel('10node.xlsx', 'Distance')
df_dis = df_dis.iloc[:,1:]
df_dis

,1,2,3,4,5,6,7,8,9,10
0,0.000000,9.219544,3.000000,7.071068,8.544004,4.472136,6.708204,3.605551,5.099020,5.000000
1,9.219544,0.000000,7.615773,5.000000,4.472136,5.000000,3.162278,5.656854,6.082763,7.071068
2,3.000000,7.615773,0.000000,7.280110,5.830952,4.123106,6.000000,3.162278,2.236068,2.000000
3,7.071068,5.000000,7.280110,0.000000,8.062258,3.162278,2.236068,4.123106,7.211103,8.062258
4,8.544004,4.472136,5.830952,8.062258,0.000000,6.082763,5.830952,6.000000,3.605551,4.242641
5,4.472136,5.000000,4.123106,3.162278,6.082763,0.000000,2.236068,1.000000,4.242641,5.000000
6,6.708204,3.162278,6.000000,2.236068,5.830952,2.236068,0.000000,3.162278,5.385165,6.324555
7,3.605551,5.656854,3.162278,4.123106,6.000000,1.000000,3.162278,0.000000,3.605551,4.242641
8,5.099020,6.082763,2.236068,7.211103,3.605551,4.242641,5.385165,3.605551,0.000000,1.000000
9,5.000000,7.071068,2.000000,8.062258,4.242641,5.000000,6.324555,4.242641,1.000000,0.000000


In [6]:
# transportation cost

shipping_cost = {(customer, facility): df_dis.iloc[customer, facility] * 10
            for customer in range(0, 10)
            for facility in range(0, 10)}

print("Number of viable pairings: {0}".format(len(shipping_cost.keys())))


Number of viable pairings: 100


In [7]:
m = gp.Model("Facility location")

Restricted license - for non-production use only - expires 2023-10-25


In [8]:
# Decision variables: facilities open or close
fact = m.addVars(10, vtype=GRB.BINARY, name='fact')

In [9]:
# Decision variables: assign customer clusters to a facility location
cartesian_prod = list(product(range(0, 10), range(0, 10)))
cust = m.addVars(cartesian_prod, lb = 0 ,vtype = GRB.CONTINUOUS, name='cust')

In [10]:
# Objective Function
# Minimize total cost
m.setObjective(gp.quicksum(df_demand[customer] * shipping_cost[customer, facility] * cust[customer, facility] for customer in range(0, 10) for facility in range(0, len(fact))) + gp.quicksum(fact[facility] for facility in range(0, len(fact))) * 200 , GRB.MINIMIZE)

# Constration
# 1. 
m.addConstrs((gp.quicksum(cust[(customer, facility)] for facility in range(0, 10)) == 1 for customer in range(0, 10)), name='Demand')

# 2.
m.addConstrs((cust[customer, facility] <= fact[facility] for customer, facility in cartesian_prod), name='Setup2ship')

m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 110 rows, 110 columns and 300 nonzeros
Model fingerprint: 0x5f8380a0
Variable types: 100 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 110 rows, 110 columns, 300 nonzeros
Variable types: 100 continuous, 10 integer (10 binary)
Found heuristic solution: objective 2000.0000000

Root relaxation: objective 1.940000e+03, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1940.0000000 1940.00000  0.00%     -    0s

Explored 1 nodes (10 simplex iterations) in 0.03 seconds (0.00 wo

In [11]:
# display optimal values of decision variables

for facility in fact.keys():
    if (abs(fact[facility].x) > 1e-6):
        print(f"\n Build a factory at location {facility + 1}.")


 Build a factory at location 1.

 Build a factory at location 2.

 Build a factory at location 3.

 Build a factory at location 4.

 Build a factory at location 5.

 Build a factory at location 7.

 Build a factory at location 8.

 Build a factory at location 9.
